In [1]:
#worked with kat and yizhen

from utils import*
import pandas as pd
import altair as alt


from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans

//anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
//anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [16]:
data = pd.read_csv('historical_projections.csv')
data

,Player,Position,ID,Draft Year,Projected SPM,Superstar,Starter,Role Player,Bust
0,Karl-Anthony Towns,C,karl-anthony-towns,2015,1.030606,0.134767,0.427183,0.163083,0.274967
1,Justise Winslow,SF,justise-winslow,2015,0.875329,0.083529,0.510905,0.176767,0.228800
2,Stanley Johnson,SF,stanley-johnson,2015,0.679493,0.067800,0.423733,0.278500,0.229967
3,Jahlil Okafor,C,jahlil-okafor,2015,0.521661,0.058717,0.409900,0.235533,0.295850
4,D`Angelo Russell,PG,d-angelo-russell,2015,0.511967,0.152033,0.342283,0.096583,0.409100
...,...,...,...,...,...,...,...,...,...
1085,Terrell Lyday,SG,terrell-lyday,2001,-1.486319,0.001152,0.018303,0.261035,0.719510
1086,Brandon Wolfram,SF,brandon-wolfram,2001,-1.493460,0.000271,0.011532,0.271560,0.716636
1087,Ruben Boumtje Boumtje,C,ruben-boumtje-boumtje,2001,-1.494069,0.000000,0.020653,0.236919,0.742427
1088,Greg Stevenson,SG,greg-stevenson,2001,-1.529188,0.000000,0.019693,0.238861,0.741446


In [35]:
nRow, nCol = data.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 1090 rows and 9 columns


In [ ]:
#Bust: Greg Oden
#success: Kyle Korver


#Definitions
#Player Player name 
#Position The player's position going into the draft 
#ID The player's identification code Draft 
#Year The year the player was eligible for the NBA draft Projected 
#SPM | The model's projected statistical plus/minus over years 2-5 of the player's NBA career Superstar
#Probability of becoming a superstar player (1 per draft, SPM >= +3.3) 
#Starter | Probability of becoming a starting-caliber player (10 per draft, SPM >= +0.5) 
#Role Player | Probability of becoming a role player (25 per draft, SPM >= -1.4)
#Bust | Probability of becoming a bust (everyone else, SPM < -1.4)

In [31]:
df = data.tail(100)

greg_data = data.loc[data["Player"]== "Greg Oden"]
greg_data

,Player,Position,ID,Draft Year,Projected SPM,Superstar,Starter,Role Player,Bust
643,Greg Oden,C,greg-oden,2007,0.822608,0.126146,0.382518,0.16427,0.327066


In [33]:
kyle_data = data.loc[data["Player"]== "Kyle Korver"]
kyle_data

,Player,Position,ID,Draft Year,Projected SPM,Superstar,Starter,Role Player,Bust
907,Kyle Korver,SG,kyle-korver,2003,-0.495672,0.000904,0.21049,0.401105,0.387501


In [34]:
compareDF = greg_data.append(kyle_data)
compareDF

,Player,Position,ID,Draft Year,Projected SPM,Superstar,Starter,Role Player,Bust
643,Greg Oden,C,greg-oden,2007,0.822608,0.126146,0.382518,0.164270,0.327066
907,Kyle Korver,SG,kyle-korver,2003,-0.495672,0.000904,0.210490,0.401105,0.387501


In [42]:
alt.Chart(data).mark_point().encode(
    alt.X('Role Player'),
    alt.Y('Bust')
)

alt.Chart(...)

In [64]:
#how the Superstar value is related to Projected SPM and position--> Utilizing size as another dimension
alt.Chart(data).mark_point().encode(
    alt.X('Projected SPM', scale=alt.Scale(zero=False)),
    alt.Y('Bust', scale=alt.Scale(zero=False)),
    alt.Size('Superstar')
)

alt.Chart(...)

In [65]:
# Visualization 1 

alt.Chart(data).mark_point().encode(
    x = "Projected SPM",
    y = "Superstar",
    color = "Starter",
    tooltip=["Player", "Position","Role Player"]
).interactive()

alt.Chart(...)

In [69]:
# The scatterplot matrix, known acronymically as SPLOM, is a relatively uncommon graphical tool that uses multiple scatterplots to determine the correlation
alt.Chart(data).mark_circle().encode(
    alt.X(alt.repeat("column"), type="quantitative"),
    alt.Y(alt.repeat("row"), type="quantitative"),
    color="Position",
    tooltip=["Player"]
).properties(
    width=150,
    height=150
).repeat(
    row=["Projected SPM", "Role Player", "Starter"],
    column=["Role Player", "Superstar", "Bust"]
).interactive()

alt.RepeatChart(...)

In [68]:
def runKNN(data, x, y, k): 
    X = x.values
    Y = y.values.reshape(-1, 1)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)#stratify=Y)
    # Build a kNN classifier
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, Y_train)
    
    # Compute the quality of our predictions
    score = knn.score(X_test, Y_test)
    print("Predicts " + y.name + " with " + str(score) + " accuracy")
    print("F1 score is " + str(f1_score(Y_test, knn.predict(X_test), labels=y.unique(), average='weighted')))
    print("Chance is: " + str(1.0 / len(y.unique())))
    
    return knn

In [93]:
data["Projected SPM"] = data["Projected SPM"].astype(str)
data["Starter"] = data["Starter"].astype(float)

x = data[["Bust"]]
y = data["Position"]

## Run kNN with one neighbor
#runKNN(df, x, y, 1)

#  five neighbors
#runKNN(df, x, y, 5)

# run it with many 
for k in range(1, 5): 
    print(str(k) + " Neighbors: ")
    runKNN(data, x, y, k)
    
y.value_counts()

1 Neighbors: 
Predicts Position with 0.1743119266055046 accuracy
F1 score is 0.16770119492069743
Chance is: 0.2
2 Neighbors: 
Predicts Position with 0.16055045871559634 accuracy
F1 score is 0.13228302024173585
Chance is: 0.2
3 Neighbors: 
Predicts Position with 0.20642201834862386 accuracy
F1 score is 0.17396505778925667
Chance is: 0.2
4 Neighbors: 
Predicts Position with 0.20642201834862386 accuracy
F1 score is 0.19452520162019077
Chance is: 0.2


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys


SG    268
PF    258
PG    220
SF    216
C     128
Name: Position, dtype: int64

In [107]:
model = runKMeans(data, 7)

# What are the stats of our prototypical players?
# These correspond to the cluster centers (the means in k-Means)
#print(data.columns)
#print(model.cluster_centers_[:100])

# (Added after class) The code below will print them out a little 
# more neatly
features = data.select_dtypes(include="number")
for c in range(0, len(model.cluster_centers_)):
    print("***** Cluster " + str(c))
    for f in range(0, len(features.columns)):
        feature = features.columns[f]
        print("Mean " + feature + " = " + str(model.cluster_centers_[c][f]))

ValueError: Length of passed values is 1090, index implies 100.